In [13]:
import pandas as pd 
import wandb
from tqdm import tqdm
import requests
import os
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
import matplotlib.pyplot as plt
import scipy.stats as stats
import matplotlib.colors as mcolors

In [42]:
from read_wandb import wandb_results
api = wandb_results("NLP2024_PROJECT_316137371_314968595", wandb_username="oribar18")

BASE_METRIC = "accuracy_per_mean_user_and_bot"

[1, 2]


In [15]:
# def result_metric(sweeps, group_name, drop_list=[0], drop_HPT=False, metric=BASE_METRIC, epoch="best"):
#     df = api.get_sweeps_results(sweeps, metric=metric) 
# 
#     config_cols = [c for c in df.columns if "config_" in c and c!="config_wandb_run_id" and c!="config_online_simulation_size"]
#     HPT_cols = [col for col in config_cols if df[col].nunique() > 1]
#     print(HPT_cols)
#     if drop_HPT:
#         df=df.drop([c for c in HPT_cols if not c in ["config_LLM_SIM_SIZE", "config_seed"]], axis=1)
#         HPT_cols = ["config_LLM_SIM_SIZE", "config_seed"]
# 
#     grouped = df.groupby([c for c in HPT_cols if c != "config_seed"])
# 
#     mean_df = grouped.mean()
#     std_df = grouped.std()
#     if epoch=="best":
#         best_col = mean_df[[c for c in mean_df.columns if (metric in c and metric[-4:] == c.split("_epoch")[0][-4:])]].idxmax(axis=1)
#     else:
#         best_col = mean_df[[c for c in mean_df.columns if f"{metric}_epoch{epoch}" in c]].idxmax(axis=1)
# 
#     result = grouped.apply(lambda x: x[best_col.loc[x.name]].values)
#     means = grouped.apply(lambda x: x[best_col.loc[x.name]].mean())
#     stds = grouped.apply(lambda x: x[best_col.loc[x.name]].std())
# 
# 
#     df_cols = {'mean': means, 'std': stds, 'values': result.values}
#     if epoch == "best": df_cols['epoch'] = best_col.apply(lambda x: int(x.split("epoch")[1]) if "epoch" in x else "last")
# 
#     df_cols['CI'] = result.apply(lambda x: bootstrap_ci(x))
# 
#     summary_df = pd.DataFrame(df_cols, index=best_col.index)
#     for d in drop_list:
#         if d in summary_df.index:
#             summary_df=summary_df.drop(d)
#     if len(summary_df.index.names) == 1:
#         return summary_df.rename_axis(group_name)
#     else:
#         return summary_df
# 
# def bootstrap_ci(data, n_bootstrap=1000, ci=0.95):
#     bootstrapped_means = []
#     for _ in range(n_bootstrap):
#         sample = np.random.choice(data, size=len(data), replace=True)
#         bootstrapped_means.append(np.mean(sample))
#     lower_bound = np.percentile(bootstrapped_means, (1 - ci) / 2 * 100)
#     upper_bound = np.percentile(bootstrapped_means, (1 + ci) / 2 * 100)
#     return lower_bound, upper_bound

In [38]:
def result_metric(sweeps, group_name, drop_list=[0], drop_HPT=False, metric=BASE_METRIC, epoch="best"):
    df = api.get_sweeps_results(sweeps, metric=metric)

    config_cols = [c for c in df.columns if "config_" in c and c!="config_wandb_run_id" and c!="config_online_simulation_size"]
    HPT_cols = [col for col in config_cols if df[col].nunique() > 1]
    print(HPT_cols)
    if drop_HPT:
        df=df.drop([c for c in HPT_cols if not c in ["config_LLM_SIM_SIZE", "config_seed"]], axis=1)
        HPT_cols = ["config_LLM_SIM_SIZE", "config_seed"]

    # Remove non-numeric columns before computing mean and std
    numeric_cols = df.select_dtypes(include=np.number).columns
    df_numeric = df[numeric_cols]

    grouped = df_numeric.groupby([c for c in HPT_cols if c != "config_seed"])

    mean_df = grouped.mean()
    std_df = grouped.std()
    
    # Re-add non-numeric columns before computing best_col
    for col in config_cols:
        if col not in mean_df.columns:
            mean_df[col] = df[col]

    if epoch=="best":
        best_col = mean_df[[c for c in mean_df.columns if (metric in c and metric[-4:] == c.split("_epoch")[0][-4:])]].idxmax(axis=1)
    else:
        best_col = mean_df[[c for c in mean_df.columns if f"{metric}_epoch{epoch}" in c]].idxmax(axis=1)

    result = grouped.apply(lambda x: x[best_col.loc[x.name]].values)
    means = grouped.apply(lambda x: x[best_col.loc[x.name]].mean())
    stds = grouped.apply(lambda x: x[best_col.loc[x.name]].std())


    df_cols = {'mean': means, 'std': stds, 'values': result.values}
    if epoch == "best": df_cols['epoch'] = best_col.apply(lambda x: int(x.split("epoch")[1]) if "epoch" in x else "last")

    df_cols['CI'] = result.apply(lambda x: bootstrap_ci(x))

    summary_df = pd.DataFrame(df_cols, index=best_col.index)
    for d in drop_list:
        if d in summary_df.index:
            summary_df=summary_df.drop(d)
    if len(summary_df.index.names) == 1:
        return summary_df.rename_axis(group_name)
    else:
        return summary_df

def bootstrap_ci(data, n_bootstrap=1000, ci=0.95):
    bootstrapped_means = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(data, size=len(data), replace=True)
        bootstrapped_means.append(np.mean(sample))
    lower_bound = np.percentile(bootstrapped_means, (1 - ci) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_means, (1 + ci) / 2 * 100)
    return lower_bound, upper_bound


# For HyperParameterTuning

For every configuration that you test in the sweep, you will receive in the table the average, standard deviation, all the values obtained for the different seed values, and also the confidence interval within which the result is located at a confidence level of 95%.

When epoch="best" is defined, you can check in which epoch the best result is obtained. If epoch=5 is defined, you will receive the result obtained for epoch number 5.

You can test multiple sweeps simultaneously by entering them into the list found in the first element of the function result_metric.

In [43]:
# Create the directory if it doesn't exist
directory = 'sweeps_csvs'
if not os.path.exists(directory):
    os.makedirs(directory)
    
sweep_results = result_metric(["rx97vkua"], "threshold", drop_HPT=False, epoch="best")
sweep_results

Total number of sweeps: 1
Download sweep_id='rx97vkua' data...


100%|██████████| 6/6 [00:00<00:00, 1754.45it/s]


['config_seed', 'config_threshold']


C:\Users\oriba\AppData\Local\Temp\ipykernel_10676\593056651.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = grouped.apply(lambda x: x[best_col.loc[x.name]].values)
C:\Users\oriba\AppData\Local\Temp\ipykernel_10676\593056651.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  means = grouped.apply(lambda x: x[best_col.loc[x.name]].mean())
C:\Users\oriba\AppData\Local\Temp\ipykernel_10676\59305

,mean,std,values,epoch,CI
threshold,,,,,
0.60,0.834485,0.002400,"[0.832787967878214, 0.8361822061805843]",last,"(0.832787967878214, 0.8361822061805843)"
0.65,0.833039,0.001341,"[0.8320904992538463, 0.8339865181465557]",last,"(0.8320904992538463, 0.8339865181465557)"
0.70,0.832000,0.001506,"[0.8330646677343833, 0.8309351584495499]",12,"(0.8309351584495499, 0.8330646677343833)"


# Result for a specific epoch

In [5]:
sweep_results = result_metric(["px13bj47"], "Hidden dim", drop_HPT=False, epoch=1)
sweep_results

Total number of sweeps: 1
Download sweep_id='px13bj47' data...
['config_seed', 'config_layers', 'config_hidden_dim']


mean       std  \
config_layers config_hidden_dim                       
2             32                 0.822903  0.004485   
              64                 0.821078  0.004435   
4             32                 0.821501  0.003652   
              64                 0.825733  0.002764   

                                                                            values  \
config_layers config_hidden_dim                                                      
2             32                 [0.8202714466298694, 0.8280817740751573, 0.820...   
              64                 [0.8224877120590862, 0.8246368273885568, 0.816...   
4             32                 [0.8179865378516612, 0.8252760405650429, 0.821...   
              64                 [0.8226790369866819, 0.8280626878370055, 0.826...   

                                                                       CI  
config_layers config_hidden_dim                                            
2             32                 (0.8202714466298694, 0.8280817740751573)  
              64                 (0.8161096617444225, 0.8246368273885568)  
4             32                 (0.8179865378516612, 0.8252760405650429)  
              64                 (0.8226790369866818, 0.8280626878370055)